# Component: Model I/O

In [16]:
!pip install langchain langchain-openai

In [17]:
from dotenv import load_dotenv
load_dotenv()

True

### PromptTemplate

In [19]:
# PromptTemplate
from langchain import PromptTemplate

template = "{product}를 홍보하기 위한 재미있고, 신선한 광고문구를 작성해 주세요."

prompt = PromptTemplate(
    template=template,
    input_variables=['product']
)

print(prompt.format(product='자동차'))

자동차를 홍보하기 위한 재미있고, 신선한 광고문구를 작성해 주세요.


In [20]:
# FewShotPromptTemplate
from langchain.prompts import FewShotPromptTemplate

examples = [
    {"question": "2 + 2는 무엇인가요?", "answer": "2 + 2 = 4"}, 
    {"question": "3 + 4는 무엇인가요?", "answer": "3 + 4 = 7"}
]

example_prompt = PromptTemplate(
    template="Q: {question}\nA: {answer}",
    input_variables=['question', 'answer']
)

fewshot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="다음 계산 문제를 해결하세요.",
    suffix="Q: {question}은 무엇인가요?\nA:",
    input_variables=['question']
)

print(fewshot_prompt.format(question='34 + 78'))

다음 계산 문제를 해결하세요.

Q: 2 + 2는 무엇인가요?
A: 2 + 2 = 4

Q: 3 + 4는 무엇인가요?
A: 3 + 4 = 7

Q: 34 + 78은 무엇인가요?
A:


In [21]:
# ChatPromptTemplate - System, Human, AI 유형별 메세지 작성
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate

sys_msg = SystemMessagePromptTemplate.from_template('당신은 친절한 챗봇입니다.')
hm_msg = HumanMessagePromptTemplate.from_template('질문: {question}')
msg = ChatPromptTemplate.from_messages([sys_msg, hm_msg])
print(msg)

msg.format_messages(question="AI를 배우려면 무엇부터 해야 하나요?")

input_variables=['question'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 친절한 챗봇입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='질문: {question}'), additional_kwargs={})]


[SystemMessage(content='당신은 친절한 챗봇입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='질문: AI를 배우려면 무엇부터 해야 하나요?', additional_kwargs={}, response_metadata={})]

### OutputParser

In [22]:
# OutputParser 인스턴트 및 PromptTemplate 인스턴스 생성
from langchain_openai import ChatOpenAI
from langchain.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

prompt_tpl = PromptTemplate(
    template="{subject} 5개의 팀을 알려주세요.\n형식 지정: {format}",
    input_variables=['subject'],    # 사용자 입력 변수
    partial_variables={'format': format_instructions}   # 고정 설정 변수
)

In [23]:
# 프롬프트 생성
query = "한국의 프로야구팀"
prompt = prompt_tpl.format(subject=query)
print(prompt)

한국의 프로야구팀 5개의 팀을 알려주세요.
형식 지정: Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [24]:
# open ai api를 사용하기 위한 인스턴스 생성 및 요청 처리
model = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0,
    max_tokens=2048
)

response = model.invoke(prompt) # prompt의 질의를 보냄

In [25]:
print(response.content)
print(type(response.content))   # string

두산 베어스, LG 트윈스, 삼성 라이온즈, 키움 히어로즈, NC 다이노스
<class 'str'>


In [26]:
print(output_parser.parse(response.content))
print(type(output_parser.parse(response.content)))  # string -> list 

['두산 베어스', 'LG 트윈스', '삼성 라이온즈', '키움 히어로즈', 'NC 다이노스']
<class 'list'>


### HuggingFace Model

In [27]:
!pip install langchain_huggingface

In [29]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

end_point = HuggingFaceEndpoint(
    repo_id="Bllossom/llama-3.2-Korean-Bllossom-3B",
    task="text-generation",
    max_new_tokens=1024
)

hf_model = ChatHuggingFace(
    llm=end_point,
    verbose=True
)

hf_model.invoke('저는 아침으로 사과를 먹었습니다. 저는 아침에 뭘 먹었을까요?')

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


AIMessage(content='당시 교등imizeBoxčná�irost Correctionsmor…طsa(apple)는 사과로, 저는 아침에 사과를 먹었습니다.', additional_kwargs={}, response_metadata={'token_usage': ChatCompletionOutputUsage(completion_tokens=31, prompt_tokens=98, total_tokens=129), 'model': '', 'finish_reason': 'stop'}, id='run-5657124d-f258-427b-bfda-ffda6edd33a0-0')

In [31]:
from langchain.model_laboratory import ModelLaboratory

model_lab = ModelLaboratory.from_llms([model, hf_model])
model_lab.compare('아침에 사과를 먹는 것의 효과를 알려주세요.')

Input:
아침에 사과를 먹는 것의 효과를 알려주세요.

client=<openai.resources.chat.completions.completions.Completions object at 0x000001BC0EC45A00> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001BC0EC56850> root_client=<openai.OpenAI object at 0x000001BC0EB5E8B0> root_async_client=<openai.AsyncOpenAI object at 0x000001BC0EC45AC0> model_name='gpt-4o-mini' temperature=0.0 model_kwargs={} openai_api_key=SecretStr('**********') max_tokens=2048
아침에 사과를 먹는 것은 여러 가지 건강상의 이점을 제공합니다. 다음은 그 효과들입니다:

1. **영양소 공급**: 사과는 비타민 C, 식이섬유, 항산화 물질이 풍부하여 면역력을 높이고 건강한 피부를 유지하는 데 도움을 줍니다.

2. **소화 개선**: 사과에 포함된 식이섬유는 소화를 촉진하고 장 건강을 개선하는 데 도움을 줄 수 있습니다. 특히, 사과의 펙틴은 장내 유익한 세균의 성장을 도와줍니다.

3. **체중 관리**: 사과는 칼로리가 낮고 식이섬유가 풍부하여 포만감을 주기 때문에 체중 관리에 도움이 될 수 있습니다. 아침에 사과를 먹으면 과식을 예방하는 데 도움이 됩니다.

4. **혈당 조절**: 사과는 혈당 수치를 안정시키는 데 도움을 줄 수 있는 저당 지수를 가지고 있습니다. 이는 아침에 에너지를 지속적으로 공급하는 데 유리합니다.

5. **심혈관 건강**: 사과에 포함된 항산화 물질과 식이섬유는 심혈관 건강을 증진시키고, 나쁜 콜레스테롤 수치를 낮추는 데 도움을 줄 수 있습니다.

6. **정신적 집중력 향상

HfHubHTTPError: 424 Client Error: Failed Dependency for url: https://router.huggingface.co/hf-inference/models/Bllossom/llama-3.2-Korean-Bllossom-3B/v1/chat/completions (Request ID: Root=1-6806dfc2-01855de17c9d292c4a579e5d;9fbb9ad5-d1a3-4ca8-a865-6f037f30b80a)

Request failed during generation: Server error: 'list' object has no attribute 'get_seq_length'